In [2]:
# Lyapunov_2d_eq.py
#
# This code constructs and trains the neural network for Example (7.1) with
# loss function (6.4) in the paper "Computing Lyapunov functions using deep
# neural networks" by Lars Grüne. The code saves the resulting network to
# the file "Lyapunov_2d_eq.h5" and plots the (flawed) Lyapunov function
#
# (C) Lars Grüne 2020

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

from tensorflow import keras

import tensorflow.keras.backend as kb

import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d

import time

In [3]:
###### Auxiliary functions ######

# define the vector field
def vf(x):
    y = [-x[:,0]-10.*x[:,1]**2,-2.*x[:,1]]
    return y

# define the part of the loss function implementing the boundary condition
@tf.function
def bound_loss(y_pred,ubound,lbound,zdata):
    lower_loss = kb.sum(kb.square(kb.min([y_pred-lbound,zdata],0)))
    upper_loss = kb.sum(kb.square(kb.max([y_pred-ubound,zdata],0)))
    custom_loss= lower_loss + upper_loss
    return custom_loss

# define the part of the loss function implementing the PDE
@tf.function
def grad_loss_eq(gradx, x_batch_train, vf_batch_train, zdata):
    g_loss = kb.sum(kb.square( kb.sum(gradx*vf_batch_train,axis=1) + kb.sum(kb.square(x_batch_train),axis=1)))
    return g_loss

# define the upper bound for the boundary condition
def upperbound ( data ):
    return 10.*data[:,0]**2 + 10.*data[:,1]**2

# define the lower bound for the boundary condition
def lowerbound ( data ):
    return 0.1*data[:,0]**2 + 0.1*data[:,1]**2


####### Construct the neural network model #######

from tensorflow.keras import layers

time1 = time.perf_counter()

model = tf.keras.Sequential()

# Dimensions and number of subsystems
inputdim = 2
subdim = 1
subnum = 2
sublayersize = 128

# Define inputs
inputs = keras.Input(shape=(inputdim,), name = 'state')


# define layer for coordinate transformation
xc = []
basename = 'coord_trafo_'

for i in range(subnum):
    thisname = basename + str(i)
    xc.append(layers.Dense(subdim, activation='linear', name = thisname)(inputs))


# define sublayers of dimension sublayersize for approximation of the
# \hat V_i in the compositional form (3.1) of the Lyapunov function
xs = []
basename = 'subsystem_'

for i in range(subnum):
    thisname = basename + str(i)
    xs.append(layers.Dense(sublayersize, activation='softplus', name = thisname)(xc[i]))


# concatenate the sublayers to compute the scalar output W
x = layers.concatenate(xs)

output = layers.Dense(1, activation='linear', name = 'Lyapunov_function')(x)

# compile the model and print summary
model = keras.Model(inputs=inputs, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 state (InputLayer)             [(None, 2)]          0           []                               
                                                                                                  
 coord_trafo_0 (Dense)          (None, 1)            3           ['state[0][0]']                  
                                                                                                  
 coord_trafo_1 (Dense)          (None, 1)            3           ['state[0][0]']                  
                                                                                                  
 subsystem_0 (Dense)            (None, 128)          256         ['coord_trafo_0[0][0]']          
                                                                                              

2022-05-30 17:38:37.848208: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
